In [ ]:
#https://www.christianhaller.me/blog/projectblog/2020-07-07-How-to-train-your-NLP-Chatbot/
#https://github.com/ChristianHallerX/DataScienceProjects/tree/master/Data/chatbot_data
#https://signal.onepointltd.com/post/102ge3r/training-a-question-answering-nlp-model
#https://medium.com/axinc-ai/convert-keras-models-to-tensorflow-lite-e654994fb93c
#https://towardsdatascience.com/how-to-build-your-own-chatbot-using-deep-learning-bb41f970e281

In [41]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
import json
import pickle

import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:


import pandas as pd

train_json = pd.read_json('/content/sample(1).json')

train_json
pd.set_option('display.max_colwidth', 300)
train_json.style.set_properties(**{'text-align': 'left'}).set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])



,intents
0,"{'tags': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'response': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}"
1,"{'tags': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'response': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}"
2,"{'tags': 'thanks', 'patterns': ['Thanks', 'Thank you', ""That's helpful"", 'Awesome, thanks', 'Thanks for helping me'], 'response': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}"
3,"{'tags': 'noanswer', 'patterns': [], 'response': [""Sorry, can't understand you"", 'Please give me more info', 'Not sure I understand'], 'context': ['']}"
4,"{'tags': 'program meanning', 'patterns': ['Meaning of Program and Course?', ' What does the course means?', 'What is a program?'], 'response': 'B.Com is a Program; Introduction to Accounting is a Course (Subject)'}"
5,"{'tags': 'course meanning', 'patterns': ['Meaning of Course?', 'What does the course means?', 'What is a program?'], 'response': 'B.Tech is a Program; Introduction to Finance is a Course (Subject)'}"
6,"{'tags': 'meanning course', 'patterns': ['Meaning of Course?', 'What does the course means?', 'What is a program?'], 'response': 'M.Tech is a Program; Introduction to Programming is a Course (Subject)'}"
7,"{'tags': 'meanning program', 'patterns': ['Meaning of Program?', 'What does the course means?', 'What is a program?'], 'response': 'Phd is a Program; Introduction to teaching is a Course (Subject)'}"
8,"{'tags': 'documents btech', 'patterns': ['What documents are required to be submitted for admission in B.tech?', ' Documents for B.Tech? ', 'B. Tech Documents Required? ', 'Required Documentation B.Tech?'], 'response': 'You need to submit color copies of Grade Sheet of 10th, 12th and adharcard/passport'}"
9,"{'tags': 'btech documents', 'patterns': ['What documents are required to be submitted for admission in B.tech?', ' Documents for B.Tech? ', 'B. Tech Documents Required? ', 'Required Documentation B.Tech?'], 'response': 'Grade Sheet of 10th, 12th and adharcard/passport (colour copies)'}"


In [45]:
with open('/content/sample(1).json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
  # print(intent)
  for pattern in intent['patterns']:
    # print(pattern)
    training_sentences.append(pattern)
    print("trainning_sentences::",pattern)
    training_labels.append(intent['tags'])
    print("trainning_labels::",intent['tags'])
    responses.append(intent['response'])
    print("response::",intent['response'])
  if intent['tags'] not in labels:
      labels.append(intent['tags'])
      print("labels::",intent['tags'])
    
    
# for intent in data['intents']:
#     for pattern in intent['patterns']:
#         training_sentences.append(pattern)
#         training_labels.append(intent['tag'])
#         responses.append(intent['responses'])
    
#     if intent['tag'] not in labels:
#         labels.append(intent['tag'])
        
num_classes = len(labels)
num_classes

trainning_sentences:: Hi there
trainning_labels:: greeting
response:: ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?']
trainning_sentences:: How are you
trainning_labels:: greeting
response:: ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?']
trainning_sentences:: Is anyone there?
trainning_labels:: greeting
response:: ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?']
trainning_sentences:: Hey
trainning_labels:: greeting
response:: ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?']
trainning_sentences:: Hola
trainning_labels:: greeting
response:: ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?']
trainning_sentences:: Hello
trainning_labels:: greeting
response:: ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?']
trainning_sentences:: Good day
trainning_labels:: greeting
response:: ['Hello, than

218

In [50]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [52]:
vocab_size = 2000
embedding_dim = 75
# original=16
max_len = 80
oov_token = "<OOV>"

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, truncating='post', maxlen=max_len)

In [57]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(120, activation='relu'))
model.add(tf.keras.layers.Dense(150, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(225, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='RMSprop', metrics=['accuracy'])

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 80, 75)            150000    
                                                                 
 global_average_pooling1d_7   (None, 75)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_40 (Dense)            (None, 120)               9120      
                                                                 
 dense_41 (Dense)            (None, 150)               18150     
                                                                 
 dense_42 (Dense)            (None, 256)               38656     
                                                                 
 dense_43 (Dense)            (None, 225)               57825     
                                                     

In [58]:
epochs = 500
batch_size=1
history = model.fit(padded_sequences, np.array(training_labels),batch_size=batch_size, epochs=epochs)

Epoch 1/500
1003/1003 [==============================] - 4s 3ms/step - loss: 5.3880 - accuracy: 0.0050
Epoch 2/500
1003/1003 [==============================] - 4s 4ms/step - loss: 5.1891 - accuracy: 0.0030
Epoch 3/500
1003/1003 [==============================] - 4s 4ms/step - loss: 5.0007 - accuracy: 0.0050
Epoch 4/500
1003/1003 [==============================] - 4s 4ms/step - loss: 4.9780 - accuracy: 0.0040
Epoch 5/500
1003/1003 [==============================] - 4s 4ms/step - loss: 4.8924 - accuracy: 0.0110
Epoch 6/500
1003/1003 [==============================] - 4s 4ms/step - loss: 4.8023 - accuracy: 0.0150
Epoch 7/500
1003/1003 [==============================] - 4s 4ms/step - loss: 4.6658 - accuracy: 0.0199
Epoch 8/500
1003/1003 [==============================] - 4s 4ms/step - loss: 4.6041 - accuracy: 0.0160
Epoch 9/500
1003/1003 [==============================] - 4s 4ms/step - loss: 4.5709 - accuracy: 0.0199
Epoch 10/500
1003/1003 [==============================] - 3s 3ms/step - l

In [40]:

model.save('chatbot_model.h5', history)

print("model created")

import pickle

# to save the fitted tokenizer
with open('tokenizer_chatbot.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder_chatbot.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

model created


In [22]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder


import random
import pickle

with open("/content/sample(1).json") as file:
    data = json.load(file)



model = keras.models.load_model('chatbot_model.h5')

    # load tokenizer object
with open('/content/tokenizer_chatbot.pickle', 'rb') as handle:
      tokenizer = pickle.load(handle)

    # load label encoder object
with open('/content/label_encoder_chatbot.pickle', 'rb') as enc:
      lbl_encoder = pickle.load(enc)

inp="Is the Degree acquired through online mode equivalent to on-campus mode of learning?"
inp=inp.lower()
max_len = 20

result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
tag = lbl_encoder.inverse_transform([np.argmax(result)])

for i in data['intents']:
        if i['tags'] == tag:
          print(i['response'])

Yes. Online-campus degree is equivalent to on-campus degree


OSError: ignored

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)


In [ ]:
tflmodel = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmp1tvwiq35/assets


In [ ]:
file = open( 'yourmodel.tflite' , 'wb' ) 


In [ ]:
file.write( tflmodel )

409588

In [ ]:
interpreter = tf.lite.Interpreter(model_path="/content/yourmodel.tflite")


In [ ]:
interpreter.allocate_tensors()


In [ ]:
input_details = interpreter.get_input_details()
input_details

[{'dtype': numpy.float32,
  'index': 0,
  'name': 'serving_default_dense_6_input:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([  1, 183], dtype=int32),
  'shape_signature': array([ -1, 183], dtype=int32),
  'sparsity_parameters': {}}]

In [ ]:
output_details = interpreter.get_output_details()
output_details

[{'dtype': numpy.float32,
  'index': 19,
  'name': 'StatefulPartitionedCall:0',
  'quantization': (0.0, 0),
  'quantization_parameters': {'quantized_dimension': 0,
   'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32)},
  'shape': array([ 1, 56], dtype=int32),
  'shape_signature': array([-1, 56], dtype=int32),
  'sparsity_parameters': {}}]